In [88]:
import pandas as pd
import requests

exchange_apikey = 'zENa4h595GT3eaCh4TfP5XQBDrHUtQCo'

In [96]:
cavpath = '../CleaningGDPdata/Cleaned_GDP_without2021&2022.csv'
df = pd.read_csv(cavpath)
df.head()

,Unnamed: 0,Country,Year,Unit Code,Value
0,0,Australia,2016,AUD,1758828.0
1,1,Australia,2017,AUD,1842635.0
2,2,Australia,2018,AUD,1946613.0
3,3,Australia,2019,AUD,1979471.0
4,4,Australia,2020,AUD,2080419.0


In [97]:
del df['Unnamed: 0']
df.head()

,Country,Year,Unit Code,Value
0,Australia,2016,AUD,1758828.0
1,Australia,2017,AUD,1842635.0
2,Australia,2018,AUD,1946613.0
3,Australia,2019,AUD,1979471.0
4,Australia,2020,AUD,2080419.0


In [114]:
unit_list = df['Unit Code'].unique().tolist()

In [62]:
# https://api.apilayer.com/exchangerates_data/convert?to={to}&from={from}&amount={amount}
exchange_rate = []
headers= {
      "apikey": "zENa4h595GT3eaCh4TfP5XQBDrHUtQCo"
    }
for i in unit_list:
    url =f'https://api.apilayer.com/exchangerates_data/convert?to=GBP&from={i}&amount=1' 
    response = requests.get(url, headers = headers).json()
    try:
        result = response['result']
        exchange_rate.append(result)
    except KeyError:
        print(f'{i} is not found, Skipping...')

In [85]:
exchange_df = pd.DataFrame({'Unit Code': unit_list,
               'Exchange rate': exchange_rate
                })
exchange_df.head()

,Unit Code,Exchange rate
0,AUD,0.546956
1,EUR,0.884453
2,CAD,0.599927
3,CZK,0.037381
4,DKK,0.118804


In [107]:
new_df = pd.merge(df, exchange_df, on = 'Unit Code', how = 'left')
new_df.head()

,Country,Year,Unit Code,Value,Exchange rate
0,Australia,2016,AUD,1758828.0,0.546956
1,Australia,2017,AUD,1842635.0,0.546956
2,Australia,2018,AUD,1946613.0,0.546956
3,Australia,2019,AUD,1979471.0,0.546956
4,Australia,2020,AUD,2080419.0,0.546956


In [108]:
new_df['GBP_Value'] = new_df['Value'] * new_df['Exchange rate']
new_df.head()

,Country,Year,Unit Code,Value,Exchange rate,GBP_Value
0,Australia,2016,AUD,1758828.0,0.546956,9.620015e+05
1,Australia,2017,AUD,1842635.0,0.546956,1.007840e+06
2,Australia,2018,AUD,1946613.0,0.546956,1.064712e+06
3,Australia,2019,AUD,1979471.0,0.546956,1.082684e+06
4,Australia,2020,AUD,2080419.0,0.546956,1.137898e+06


In [112]:
new_df['GBP_Value'] = new_df['GBP_Value'].astype(int)

In [113]:
new_df.head()

,Country,Year,Unit Code,Value,Exchange rate,GBP_Value
0,Australia,2016,AUD,1758828.0,0.546956,962001
1,Australia,2017,AUD,1842635.0,0.546956,1007840
2,Australia,2018,AUD,1946613.0,0.546956,1064711
3,Australia,2019,AUD,1979471.0,0.546956,1082683
4,Australia,2020,AUD,2080419.0,0.546956,1137897


In [115]:
new_df.to_csv("newCleaned_GDP_with2021&2022.csv")

In [117]:
new_df = new_df.loc[new_df['Year'] < 2021]
new_df.to_csv("newCleaned_GDP_without2021&2022.csv")